In [6]:
import mysql.connector
import json, os, re
from datetime import datetime
import pandas as pd
import re
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\Feb-25")
from query import *
# conn = mysql.connector.connect(
#     host="172.22.225.155",
#     user="cog_mf",
#     password="bnYwFChjLAV2Z%9E",
#     database="cog_mf",
#     port=3306
# )
# cursor = conn.cursor()
# print("Successful Connection !!")
# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="1234",
#     database="cog_updated_db"
# )
def get_json_paths(root_folder):
    json_paths = []
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isfile(folder_path) and folder_path.endswith(".json"):
            json_paths.append(folder_path)
    return json_paths

def load_all_records(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
                records = data.get("records", [])
                name = " ".join(records[0].get("value", {}).get("amc_name", "").split(" ")[:2])
                if name not in all_records:
                    all_records[name] = records
                else:
                    name += " 2"
                    all_records[name] = records
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records

def load_all_records_mydata(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)  # The whole dict is the data
                sample_scheme = next(iter(data.values()))
                amc_name = sample_scheme.get("amc_name", "").strip()
                name = " ".join(amc_name.split(" ")[:2])
                
                if name not in all_records:
                    all_records[name] = data
                else:
                    name += " 2"
                    all_records[name] = data
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records


In [22]:
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\FEB25DATA") #FEB25DATA
json_paths = get_json_paths(base_dir)
combined_records = load_all_records_mydata(json_paths)

In [ ]:
# mydata
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="cog_updated_db"
)

# conn = mysql.connector.connect(
#     host="172.22.225.155",
#     user="cog_mf",
#     password="bnYwFChjLAV2Z%9E",
#     database="cog_mf",
#     port=3306,
#     charset='utf8mb4'
# )
cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM mf_amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO mf_amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid

def insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month, data_from):
    keys = ["amc_id", "entered_time", "amc_for_month", "data_from", "amc_name", "benchmark_index",
            "main_scheme_name", "mutual_fund_name", "monthly_aaum_date", "monthly_aaum_value",
            "scheme_launch_date", "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple"]

    query = f"INSERT INTO mf_mutual_funds ({', '.join(keys)}) VALUES ({', '.join(['%s'] * len(keys))})"
    values = [
        amc_id, curr_time, amc_month, data_from, details.get("amc_name", ""),
        details.get("benchmark_index", ""),
        details.get("main_scheme_name", ""),
        details.get("mutual_fund_name", ""),
        details.get("monthly_aaum_date", ""),
        details.get("monthly_aaum_value", ""),
        details.get("scheme_launch_date", ""),
        details.get("min_addl_amt", ""),
        details.get("min_addl_amt_multiple", ""),
        details.get("min_amt", ""),
        details.get("min_amt_multiple", "")
    ]
    try:
        cursor.execute(query, values)
    except Exception as e:
        print("Error inserting mutual fund:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Query:", query)
        print("Values:", values)
        print("Exception:", e)
        return None
    
    return cursor.lastrowid

def insert_fund_managers(cursor, amc_id,entered_time, amc_for_month,data_from, mutual_fund_id, details):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_fund_managers
            (amc_id,entered_time, amc_for_month, data_from, mutual_fund_id, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            amc_id,
            entered_time, 
            amc_for_month,
            data_from,
            mutual_fund_id,
            details["main_scheme_name"],
            manager.get("name", ""),
            manager.get("qualification", ""),
            manager.get("managing_fund_since", ""),
            manager.get("total_exp", "")
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error in Manager: {e}")

def insert_load(cursor, amc_id, mutual_fund_id, details):
    load = details.get("load", {})
    if not isinstance(load, dict):
        print(f"Error in Loads, {details.get('main_scheme_name', '')}")
        return
    query = """
        INSERT INTO mf_transformed_loads
        (amc_id, mutual_fund_id, main_scheme_name, entry_load, exit_load)
        VALUES (%s, %s, %s, %s, %s)
    """
    values = [
        amc_id,
        mutual_fund_id,
        details.get("main_scheme_name", ""),
        load.get("entry_load", ""),
        load.get("exit_load", "")
    ]
    
    try:
        cursor.execute(query, values)
    except Exception as e:
        print(f"Error in Load: {e}")

def insert_metrics(cursor, amc_id, mutual_fund_id, details):
    keys = ['amc_id', 'main_scheme_name', 'mutual_fund_id', "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity", 
            "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay", "mod_duration", "port_turnover_ratio", "r_squared_ratio", 
            "roe_ratio", "sharpe", "sortino_ratio", "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"]
    metrics = details.get("metrics", {})
    if not isinstance(metrics, dict):
        print(f"Error in Metrics, {details.get('main_scheme_name', '')}")
        return
    query = f"INSERT INTO mf_transformed_metrics ({', '.join(keys)}) VALUES ({', '.join(['%s'] * len(keys))})"
    values = [amc_id, details.get("main_scheme_name", ""), mutual_fund_id] + [metrics.get(k, "") for k in keys[3:]]
    try:
        cursor.execute(query, values)
    except Exception as e:
        print(f"Error in Metrics: {e}")

curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEBRUARY 25"
data_from = "mydata"

for amc_name, schemes in combined_records.items():
    print(f"Inserting Data For: {amc_name}")
    amc_id = get_or_create_amc(next(iter(schemes.values())).get("amc_name", ""))
    for scheme_name, details in schemes.items():
        try:
            mutual_fund_id = insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_managers(cursor, amc_id,curr_time, amc_month, data_from, mutual_fund_id, details)
                insert_load(cursor, amc_id, mutual_fund_id, details)
                insert_metrics(cursor, amc_id, mutual_fund_id, details)
            else:
                print(f"Skipped Data Insertion for AMC {scheme_name}")
        except Exception as e:
            print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")

conn.commit()
cursor.close()
conn.close()
print("Working!!")


In [25]:
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\Feb-25")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)

In [ ]:
#finkstein
conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)
cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM mf_amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO mf_amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid

def insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month,data_from):
    keys = ["amc_id", "entered_time", "amc_for_month","data_from", "amc_name", "benchmark_index",
            "main_scheme_name", "mutual_fund_name", "monthly_aaum_date", "monthly_aaum_value",
            "scheme_launch_date", "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple"]
    
    query = f"""
        INSERT INTO mf_mutual_funds ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    values = [
        amc_id, curr_time, amc_month,data_from, details.get("amc_name", ""),
        ", ".join(details.get("benchmark_index", [])),
        details.get("main_scheme_name", ""),
        details.get("mutual_fund_name", ""),
        details.get("monthly_aaum_date", ""),
        details.get("monthly_aaum_value", ""),
        details.get("scheme_launch_date", ""),
        details.get("min_addl_amt", ""),
        details.get("min_addl_amt_multiple", ""),
        details.get("min_amt", ""),
        details.get("min_amt_multiple", "")
    ]

    try:
        cursor.execute(query, values)
    except Exception as e:
        print("Error inserting mutual fund:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Query:", query)
        print("Values:", values)
        print("Exception:", e)
        return None
    return cursor.lastrowid

def insert_fund_managers(cursor, amc_id, entered_time, amc_for_month, data_from, mutual_fund_id, details):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_fund_managers
            (amc_id, entered_time, amc_for_month, data_from, mutual_fund_id, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            amc_id,
            entered_time,
            amc_for_month,
            data_from,
            mutual_fund_id,
            details["main_scheme_name"],
            manager.get("name", ""),
            manager.get("qualification", ""),
            manager.get("managing_fund_since", ""),
            manager.get("total_exp", "")
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error in Manager: {e}")

def insert_load(cursor, amc_id, mutual_fund_id, details):
    entry_load = exit_load = ""
    for item in details.get("load", []):
        if item.get("type") == "entry":
            entry_load = item.get("comment", "")
        elif item.get("type") == "exit":
            exit_load = item.get("comment", "")

    query = """
        INSERT INTO mf_transformed_loads
        (amc_id, mutual_fund_id, main_scheme_name, entry_load, exit_load)
        VALUES (%s, %s, %s, %s, %s)
    """
    values = [
        amc_id,
        mutual_fund_id,
        details.get("main_scheme_name", ""),
        entry_load,
        exit_load
    ]
    cursor.execute(query, values)

def insert_metrics(cursor, amc_id, mutual_fund_id, details):
    metric_template = {
        "alpha": "", "arithmetic_mean_ratio": "", "average_div_yld": "", "average_pb": "", "average_pe": "",
        "avg_maturity": "", "beta": "", "correlation_ratio": "", "downside_deviation": "", "information_ratio": "",
        "macaulay": "", "mod_duration": "", "port_turnover_ratio": "", "r_squared_ratio": "", "roe_ratio": "",
        "sharpe": "", "sortino_ratio": "", "std_dev": "", "tracking_error": "", "treynor_ratio": "",
        "upside_deviation": "", "ytm": ""
    }

    for metric in details.get("metrics", []):
        metric_name = metric.get("name")
        if metric_name in metric_template:
            metric_template[metric_name] = metric.get("value", "")

    keys = ["amc_id", "main_scheme_name", "mutual_fund_id"] + list(metric_template.keys())
    query = f"""
        INSERT INTO mf_transformed_metrics
        ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """
    values = [
        amc_id,
        details.get("main_scheme_name", ""),
        mutual_fund_id
    ] + list(metric_template.values())

    cursor.execute(query, values)

# Begin inserting all records
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB 25"
data_from = "finkstein"

for amc_key, records in combined_records.items():
    print(f"Doing {amc_key}")
    for record in records:
        try:
            details = record["value"]
            amc_id = get_or_create_amc(details["amc_name"])
            mutual_fund_id = insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_managers(cursor, amc_id, curr_time, amc_month, data_from, mutual_fund_id, details)
                insert_load(cursor, amc_id, mutual_fund_id, details)
                insert_metrics(cursor, amc_id, mutual_fund_id, details)
            else:
                print
        except Exception as e:
            print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")
    print(f"Done {amc_key}")
conn.commit()
cursor.close()
conn.close()
print("Data insertion completed.")


In [ ]:
# Error inserting scheme 'HYBRID EQUITY FUND PGIM INDIA' under AMC 'PGIM India': 1406 (22001): Data too long for column 'managing_fund_since' at row 1
# Error inserting scheme 'MIRAE ASSET NIFTY 100 ESG SECTOR LEADERS E NSE Symbol ESG  BSE Scrip Code 543246 Exchange Traded Fund ETF' under AMC 'Mirae Asset 2': 1406 (22001): Data too long for column 'main_scheme_name' at row 1
# Error inserting scheme 'MIRAE ASSET NIFTY EV AND NEW AGE AUTOMOTI NSE Symbol EVINDIA  BSE Scrip Code 544212 Exchange Traded Fund ETF' under AMC 'Mirae Asset 2': 1406 (22001): Data too long for column 'main_scheme_name' at row 1
# Error inserting scheme 'Motilal Oswal Balanced Advantage Fund' under AMC 'Motilal Oswal': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'Motilal Oswal Multi Asset Fund' under AMC 'Motilal Oswal': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'Motilal Oswal Arbitrage Fund' under AMC 'Motilal Oswal': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'Motilal Oswal Ultra Short Term Fund' under AMC 'Motilal Oswal': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'Motilal Oswal Liquid Fund' under AMC 'Motilal Oswal': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'Motilal Oswal Gold and Silver ETFs Fund of Funds' under AMC 'Motilal Oswal 2': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'Navi Nifty India Manu In An open ended scheme replicating  tracking Nifty India  Period Navi Niy India Manufacturing Index Fund' under AMC 'Navi AMC': 1406 (22001): Data too long for column 'main_scheme_name' at row 1
# Error inserting scheme 'SHRIRAM OVERNIGHT FUND' under AMC 'Shriram Asset': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'SHRIRAM Nifty 1D Rate Liquid ETF' under AMC 'Shriram Asset': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Error inserting scheme 'SHRIRAM LIQUID FUND' under AMC 'Shriram Asset': Failed processing format-parameters; Python 'dict' cannot be converted to a MySQL type
# Working!!

In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime
import os
import re

def sanitize_sheet_name(name):
    name = re.sub(r'[^A-Za-z0-9_ ]+', '', name)
    return name.strip().replace(" ", "_")[:31]

def fetch_data_for_source(conn, amc_name, source, entered_time_pattern="%"):
    query = f"""
    SELECT
        mf.data_from,mf.entered_time,mf.amc_for_month,mf.main_scheme_name,mf.benchmark_index,mf.monthly_aaum_value,
        mf.monthly_aaum_value,mf.scheme_launch_date,mf.min_addl_amt,mf.min_addl_amt_multiple,mf.min_amt,mf.min_amt_multiple,tl.entry_load, tl.exit_load,
        tm.alpha,tm.arithmetic_mean_ratio,tm.average_div_yld,tm.average_pb,tm.average_pe,tm.avg_maturity,tm.beta,tm.correlation_ratio,tm.downside_deviation,
        tm.information_ratio,tm.macaulay,tm.mod_duration,tm.port_turnover_ratio,tm.r_squared_ratio, tm.roe_ratio, tm.sharpe,tm.sortino_ratio,tm.std_dev,
        tm.tracking_error,tm.treynor_ratio,tm.upside_deviation,tm.ytm
    FROM mf_mutual_funds mf
    LEFT JOIN mf_transformed_loads tl ON mf.id = tl.mutual_fund_id
    LEFT JOIN mf_transformed_metrics tm ON mf.id = tm.mutual_fund_id
    WHERE mf.amc_name LIKE %s AND mf.data_from = %s AND mf.entered_time LIKE %s;
    """
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query, (f"%{amc_name}%", source, entered_time_pattern))
    rows = cursor.fetchall()
    cursor.close()
    return pd.DataFrame(rows) if rows else pd.DataFrame()

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

# cursor = conn.cursor()

amc_names = pd.read_sql("SELECT amc_name FROM mf_amcs;", conn)["amc_name"].tolist()

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_path = f"comparison_export_{timestamp}.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    for amc in amc_names:
        df_A = fetch_data_for_source(conn, amc, "finkstein","%2025-04-22%")
        df_B = fetch_data_for_source(conn, amc, "mydata","%2025-04-22%")
        
        if df_A.empty and df_B.empty:
            continue
        
        sheet_name = sanitize_sheet_name(amc)
        startrow = 0

        workbook = writer.book
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'fg_color': '#D7E4BC',
            'border': 1
        })

        # fink
        if not df_A.empty:
            # df_A.insert(0, "source", "finkstein")
            df_A.to_excel(writer, sheet_name=sheet_name, startrow=startrow, index=False)
            worksheet = writer.sheets[sheet_name]
            for col_num, value in enumerate(df_A.columns.values):
                worksheet.write(startrow, col_num, value, header_format)
                max_len = max(df_A[value].astype(str).map(len).max(), len(value))
                worksheet.set_column(col_num, col_num, max_len + 2)
            startrow += len(df_A) + 3

        # mydata
        if not df_B.empty:
            # df_B.insert(0, "source", "mydata")
            df_B.to_excel(writer, sheet_name=sheet_name, startrow=startrow, index=False)
            worksheet = writer.sheets[sheet_name]
            for col_num, value in enumerate(df_B.columns.values):
                worksheet.write(startrow, col_num, value, header_format)
                max_len = max(df_B[value].astype(str).map(len).max(), len(value))
                worksheet.set_column(col_num, col_num, max_len + 2)

print(f"Data exported to: {output_path}")
conn.close()
